### Importing Libraries

In [2]:
import torch
import cv2
import numpy as np
from ultralytics import YOLO
from deepface import DeepFace
import matplotlib.pyplot as plt
import os
from tqdm import tqdm

import numpy as np
from deepface import DeepFace

### Check if GPU Computing is Available

In [4]:
# Check if a GPU is available and set the device accordingly
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Check if CUDA is available
if torch.cuda.is_available():
    print("CUDA is available. PyTorch is using the GPU.")
    print(f"Device name: {torch.cuda.get_device_name(0)}")
else:
    print("CUDA is not available. PyTorch is using the CPU.")

CUDA is not available. PyTorch is using the CPU.


### Load Face Detection Model

In [5]:
# Load the YOLOv8 model and move it to the GPU if available
model = YOLO('yolov8n-face.pt').to(device)

### Creating Face Embeddings for the Entire Folder

In [4]:
# Path to the directory containing images
image_dir = r'G:\Computer_Vision\Data\Goa'  # Change this to your directory
input_folder_name = os.path.basename(image_dir)
output_folder_name = f"{input_folder_name}_embeddings"
meta_data_folder = os.path.join(os.path.dirname(image_dir), 'meta_data', output_folder_name)

# Create directories for saving embeddings
os.makedirs(meta_data_folder, exist_ok=True)

# Get all image files in the specified directory
image_files = [f for f in os.listdir(image_dir) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]

# Loop through all image files with a single progress bar
with tqdm(total=len(image_files), desc="Processing Images", unit="image") as pbar:
    for image_name in image_files:
        image_path = os.path.join(image_dir, image_name)

        # Load and prepare the image
        image = cv2.imread(image_path)
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert to RGB for detection

        # Perform face detection (set verbose to False if applicable)
        results = model(image_rgb, verbose=False)

        # Process results and get embeddings
        face_embeddings = []

        for result in results:
            boxes = result.boxes

            for box in boxes:
                xyxy = box.xyxy[0].cpu().numpy()  # Get coordinates as a numpy array
                conf = box.conf[0].cpu().numpy() if box.conf is not None else 0  # Extract confidence

                if conf > 0.1:  # Confidence threshold
                    x1, y1, x2, y2 = map(int, xyxy)

                    # Extract face
                    face_crop = image[y1:y2, x1:x2]

                    # Convert face crop to RGB for embedding extraction
                    face_crop_rgb = cv2.cvtColor(face_crop, cv2.COLOR_BGR2RGB)

                    # Get the embedding using DeepFace
                    try:
                        embeddings = DeepFace.represent(img_path=face_crop_rgb, model_name='Facenet', enforce_detection=False)
                        face_embeddings.append(embeddings[0]['embedding'])
                    except Exception as e:
                        print(f"Error processing detected face in {image_name}: {e}")

        # Save embeddings to a file with the original image name
        if face_embeddings:
            base_name = os.path.splitext(os.path.basename(image_path))[0]
            np.save(os.path.join(meta_data_folder, f'{base_name}.npy'), face_embeddings)

        # Update progress bar
        pbar.update(1)

print("Processing complete.")

Processing Images: 100%|██████████| 76/76 [00:49<00:00,  1.54image/s]

Processing complete.


# consistant variant

In [6]:
# Path to the directory containing images
image_dir = r'G:\Computer_Vision\Data\Goa'  # Change this to your directory
input_folder_name = os.path.basename(image_dir)
output_folder_name = f"{input_folder_name}_embeddings"
meta_data_folder = os.path.join(os.path.dirname(image_dir), 'meta_data', output_folder_name)


In [7]:
# Create directories for saving embeddings
os.makedirs(meta_data_folder, exist_ok=True)

# Get all image files in the specified directory
image_files = [f for f in os.listdir(image_dir) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]

# Define a function to preprocess face crops
def preprocess_face_crop(face_crop):
    return cv2.cvtColor(face_crop, cv2.COLOR_BGR2RGB)

# Define a function to get embeddings
def get_face_embeddings(face_crop):
    try:
        embeddings = DeepFace.represent(img_path=face_crop, model_name='Facenet', enforce_detection=False)
        return embeddings[0]['embedding']
    except Exception as e:
        print(f"Error processing face crop: {e}")
        return None

# Process images with a progress bar
with tqdm(total=len(image_files), desc="Processing Images", unit="image") as pbar:
    for image_name in image_files:
        image_path = os.path.join(image_dir, image_name)
        image = cv2.imread(image_path)
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # Perform face detection
        results = model(image_rgb, verbose=False)
        face_embeddings = []

        for result in results:
            boxes = result.boxes

            for box in boxes:
                xyxy = box.xyxy[0].cpu().numpy()
                conf = box.conf[0].cpu().numpy() if box.conf is not None else 0

                if conf > 0.1:
                    x1, y1, x2, y2 = map(int, xyxy)
                    face_crop = image[y1:y2, x1:x2]
                    face_crop_rgb = preprocess_face_crop(face_crop)
                    embedding = get_face_embeddings(face_crop_rgb)
                    if embedding is not None:
                        face_embeddings.append(embedding)

        if face_embeddings:
            base_name = os.path.splitext(os.path.basename(image_path))[0]
            np.save(os.path.join(meta_data_folder, f'{base_name}.npy'), face_embeddings)

        pbar.update(1)

print("Processing complete.")

Processing Images: 100%|██████████| 76/76 [00:45<00:00,  1.66image/s]

Processing complete.
